<a href="https://colab.research.google.com/github/muhammadghazyy/Tensorflow-Projects/blob/main/mnist_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn.preprocessing

from google.colab import files

In [ ]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c digit-recognizer
!unzip test.csv.zip
!unzip train.csv.zip

In [97]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [98]:
test.tail()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
27995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,252,20,0,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
label = train['label']
train.drop('label',inplace=True,axis=1)

In [27]:
label_bin = sklearn.preprocessing.LabelBinarizer()
labels = label_bin.fit_transform(label)
labels

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [58]:
#Hyperparameters

num_classes = len(labels[0])
img_size = (28,28,1)
epoch = 25
batch_size = 32

In [41]:
#Dividing into Train and Validation sets

xtrain = train[:int(0.8*len(train['pixel0']))]
xtrainlab = labels[:int(0.8*len(train['pixel0']))]
xval = train[int(0.8*len(train['pixel0'])):]
xvallab = labels[int(0.8*len(train['pixel0'])):]

In [44]:
#Nornmalizing the pixels

xtrain = xtrain/255.0
xval = xval/255.0

Model building using keras

In [62]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(256),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(num_classes,activation='softmax')
])
model.compile(loss ='categorical_crossentropy',optimizer = 'Adam',metrics = ['acc'])

In [63]:
history = model.fit(xtrain,xtrainlab, validation_data=(xval,xvallab),epochs=10,batch_size=32)

Epoch 1/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.4010 - acc: 0.8824 - val_loss: 0.3196 - val_acc: 0.9076
Epoch 2/10
1050/1050 [==============================] - 4s 3ms/step - loss: 0.3190 - acc: 0.9083 - val_loss: 0.3085 - val_acc: 0.9124
Epoch 3/10
1050/1050 [==============================] - 4s 3ms/step - loss: 0.3050 - acc: 0.9126 - val_loss: 0.2966 - val_acc: 0.9185
Epoch 4/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.2947 - acc: 0.9154 - val_loss: 0.3074 - val_acc: 0.9131
Epoch 5/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.2857 - acc: 0.9172 - val_loss: 0.2896 - val_acc: 0.9206
Epoch 6/10
1050/1050 [==============================] - 4s 3ms/step - loss: 0.2855 - acc: 0.9179 - val_loss: 0.2927 - val_acc: 0.9156
Epoch 7/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.2810 - acc: 0.9195 - val_loss: 0.3497 - val_acc: 0.9017
Epoch 8/10
1050/1050 [==============================] - 4s 3ms

In [65]:
test1 = test.copy()/255.0

In [ ]:
#Model prediction
ans = model.predict(test1)

In [ ]:
final = [np.argmax(an) for an in ans]

In [102]:
sub = pd.DataFrame({
    'ImageId':range(1,28001),
    'Label':final
})
sub

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,7
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


Submitting into kaggle

In [100]:
sub.to_csv('submission.csv',index=False)
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "First try"

100% 208k/208k [00:01<00:00, 124kB/s]
Successfully submitted to Digit Recognizer